In [126]:

from backtesting import Backtest, Strategy
from backtesting.test import EURUSD, SMA 
import numpy as np
import pandas as pd

In [127]:

import sys
sys.path.append('../../')

In [128]:
from config.constants import *

In [129]:
SYMBOL

{'XAUUSD': {'H4': 'XAUUSD/XAUUSD_H4 - XAUUSD_H4'}}

In [130]:
XAUUSD = pd.read_csv(PATH_DATA + SYMBOL['XAUUSD']['H4'] +'.csv',index_col='Time',parse_dates=True)
XAUUSD.index.name = None

In [131]:
start_date = '2007-01-01'
end_date = '2008-03-10'

In [132]:
XAUUSD =  XAUUSD.loc[start_date:end_date]

In [133]:
XAUUSD

,Open,High,Low,Close,Volume
2007-01-01 00:00:00,635.679,636.194,635.009,635.497,95
2007-01-01 04:00:00,635.336,636.220,635.254,635.699,97
2007-01-01 08:00:00,635.780,636.225,635.201,635.993,99
2007-01-01 12:00:00,636.053,636.189,635.117,636.006,90
2007-01-01 16:00:00,635.902,636.185,635.006,635.989,96
...,...,...,...,...,...
2008-03-10 04:00:00,975.066,976.794,974.364,975.817,236
2008-03-10 08:00:00,975.850,980.428,965.608,967.910,240
2008-03-10 12:00:00,967.982,971.957,961.151,965.800,240
2008-03-10 16:00:00,965.923,975.292,965.361,970.837,240


In [134]:
EURUSD


,Open,High,Low,Close,Volume
2017-04-19 09:00:00,1.07160,1.07220,1.07083,1.07219,1413
2017-04-19 10:00:00,1.07214,1.07296,1.07214,1.07260,1241
2017-04-19 11:00:00,1.07256,1.07299,1.07170,1.07192,1025
2017-04-19 12:00:00,1.07195,1.07280,1.07195,1.07202,1460
2017-04-19 13:00:00,1.07200,1.07230,1.07045,1.07050,1554
...,...,...,...,...,...
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065


In [135]:
EURUSD.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000 entries, 2017-04-19 09:00:00 to 2018-02-07 15:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5000 non-null   float64
 1   High    5000 non-null   float64
 2   Low     5000 non-null   float64
 3   Close   5000 non-null   float64
 4   Volume  5000 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 234.4 KB


In [136]:
XAUUSD.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1928 entries, 2007-01-01 00:00:00 to 2008-03-10 20:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1928 non-null   float64
 1   High    1928 non-null   float64
 2   Low     1928 non-null   float64
 3   Close   1928 non-null   float64
 4   Volume  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [137]:
def ATR(df, n):
    df = df.copy()
    df['High-Low'] = abs(df['High'] - df['Low'])
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-PrevClose',
                   'Low-PrevClose']].max(axis=1, skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df = df.drop(['High-Low', 'High-PrevClose', 'Low-PrevClose'], axis=1)
    return df

In [138]:
def calculate_position_size(account_size, risk_percentage, entry_price, stop_loss):
    # Calculate the dollar amount at risk per trade
    dollar_at_risk = account_size * risk_percentage / 100

    # Calculate the distance between entry price and stop-loss
    distance = entry_price - stop_loss

    # Calculate the position size
    position_size = dollar_at_risk / distance

    return position_size

In [184]:
class Momentum(Strategy) :
    def init(self):
        self.data.df['ATR'] = ATR(self.data.df, 20)['ATR']
        self.data.df['direction'] = np.where(self.data.Close > self.data.Open, 'bull', 'bear')
        self.data.df['dir_count'] = self.data.df.groupby((self.data.df['direction'] != self.data.df['direction'].shift(1)).cumsum()).cumcount()+1
    

    def next(self):
        current_dir_count = self.data.df.loc[self.data.df.index[-1], 'dir_count']
        current_direction = self.data.df.loc[self.data.df.index[-1], 'direction']
        current_ATR = self.data.df.loc[self.data.df.index[-1], 'ATR']
        
        if current_dir_count >= 5 and current_direction == 'bull' and not self.position.is_short and not self.position.is_long and not np.isnan(current_ATR):
            if self.position.is_long :
                print('Have a order')
            tp = self.data.Close[-1] + current_ATR * 0.5
            sl = self.data.Open[-1]
            limit = self.data.Close[-1]
                # Place your buy logic here
            self.buy(sl=sl, tp=tp,size=0.01)
        if current_dir_count >= 5 and current_direction == 'bear' and not self.position.is_short and not self.position.is_long and not np.isnan(current_ATR):
            tp = self.data.Close[-1] - current_ATR * 0.5
            sl = self.data.Open[-1]
            limit = self.data.Close[-1]
            self.sell(sl=sl, tp=tp ,size=0.01)  # Place your buy logic here

In [185]:
bt = Backtest(XAUUSD, Momentum, cash=1000000, margin=1/2000)
start = bt.run()
start

Start                     2007-01-01 00:00:00
End                       2008-03-10 20:00:00
Duration                    434 days 20:00:00
Exposure Time [%]                     6.53527
Equity Final [$]                  649252.3416
Equity Peak [$]                 1021347.34785
Return [%]                         -35.074766
Buy & Hold Return [%]               53.168937
Return (Ann.) [%]                  -25.309396
Volatility (Ann.) [%]                40.41729
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -54.017223
Avg. Drawdown [%]                  -28.796566
Max. Drawdown Duration      412 days 04:00:00
Avg. Drawdown Duration      207 days 00:00:00
# Trades                                   89
Win Rate [%]                        37.078652
Best Trade [%]                        0.46685
Worst Trade [%]                     -1.164448
Avg. Trade [%]                    

In [186]:
bt.plot()

Row(id='13074', ...)